In [4]:
import pyomo.environ as pyo


m = pyo.ConcreteModel()

# Set declarations (assuming that the 'orders' and 'machines' sets are based on the maximum values from the data)
m.I = pyo.Set(initialize=range(1, 8))  # Set of orders i
m.J = pyo.Set(initialize=range(1, 4))  # Set of machines j

# Parameter declarations
# Cost on Machine
m.r = pyo.Param(m.I, initialize={1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 1, 7: 2})
m.d = pyo.Param(m.I, initialize={1: 16, 2: 13, 3: 21, 4: 28, 5: 24, 6: 28, 7: 23})
m.c = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 6, (1, 3): 8,
                                                  (2, 1): 8, (2, 2): 5, (2, 3): 6,
                                                  (3, 1): 12, (3, 2): 7, (3, 3): 10,
                                                  (4, 1): 10, (4, 2): 6, (4, 3): 8,
                                                  (5, 1): 8, (5, 2): 5, (5, 3): 7,
                                                  (6, 1): 12, (6, 2): 7, (6, 3): 10,
                                                  (7, 1): 12, (7, 2): 7, (7, 3): 10})


# Durations p_im for Set 1
m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 14, (1, 3): 12,
                                               (2, 1): 6, (2, 2): 8, (2, 3): 7,
                                               (3, 1): 11, (3, 2): 16, (3, 3): 13,
                                               (4, 1): 6, (4, 2): 12, (4, 3): 8,
                                               (5, 1): 10, (5, 2): 16, (5, 3): 12,
                                               (6, 1): 7, (6, 2): 12, (6, 3): 10,
                                               (7, 1): 10, (7, 2): 8, (7, 3): 10})

# Uncomment the following and adjust the values for the second set of durations p_im for Set 2

# m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,
#                                                (2, 1): 3, (2, 2): 4, (2, 3): 3,
#                                                (3, 1): 2, (3, 2): 4, (3, 3): 3,
#                                                (4, 1): 3, (4, 2): 6, (4, 3): 4,
#                                                (5, 1): 2, (5, 2): 4, (5, 3): 3,
#                                                (6, 1): 1, (6, 2): 3, (6, 3): 2,
#                                                (7, 1): 1, (7, 2): 2, (7, 3): 1})

# Decision Variables
m.y = pyo.Var(m.I, m.J, within=pyo.Binary)  # Job assignment to machine
m.s = pyo.Var(m.I, within=pyo.NonNegativeReals)  # Start time of each job

# Objective Function: Minimize the total cost
def obj_rule(m):
    return sum(m.c[i,j] * m.y[i,j] for i in m.I for j in m.J)

m.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Assignment Constraint: Each job assigned to exactly one machine
def assignment_rule(m, i):
    return sum(m.y[i,j] for j in m.J) == 1

m.assignment_constraint = pyo.Constraint(m.I, rule=assignment_rule)

# Machine Availability and Precedence Constraint: 
# Ensures that for any two jobs i and k, if job i precedes job k, then job k can only start after job i is completed
def precedence_rule(m, i, k, j):
    if i < k:  # Assuming job order determines precedence
        return m.s[i] + sum(m.p[i,jj] * m.y[i,jj] for jj in m.J) <= m.s[k]
    else:
        return pyo.Constraint.Skip

m.precedence_constraint = pyo.Constraint(m.I, m.I, m.J, rule=precedence_rule)

# Release Date Constraint
def release_date_rule(m, i):
    return m.s[i] >= m.r[i]

m.release_date_constraint = pyo.Constraint(m.I, rule=release_date_rule)

# Due Date Constraint
def due_date_rule(m, i):
    return m.s[i] + sum(m.p[i,j] * m.y[i,j] for j in m.J) <= m.d[i]

m.due_date_constraint = pyo.Constraint(m.I, rule=due_date_rule)

solver = pyo.SolverFactory('gurobi')
solver.options['VarBranch'] = 0  # Set to use the most fractional variable branching rule
results = solver.solve(m, tee=True)

# # Print the results
# print("\nTotal Cost: ", pyo.value(m.objective))
# print("\nJob Assignment to Machine:")
# for i in m.I:
#     for j in m.J:
#         if pyo.value(m.y[i,j]) == 1:
#             print("Job", i, "assigned to Machine", j, "with start time", pyo.value(m.s[i]))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Read LP format model from file /tmp/tmp6sl37ve3.pyomo.lp
Reading time = 0.00 seconds
x1: 84 rows, 28 columns, 371 nonzeros
Set parameter VarBranch to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 84 rows, 28 columns and 371 nonzeros
Model fingerprint: 0xbee3aef8
Variable types: 7 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 8 rows and 11 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Mode

In [2]:
import pyomo.environ as pyo


m = pyo.ConcreteModel()

# Set declarations (assuming that the 'orders' and 'machines' sets are based on the maximum values from the data)
m.I = pyo.Set(initialize=range(1, 8))  # Set of orders i
m.J = pyo.Set(initialize=range(1, 4))  # Set of machines j

# Parameter declarations
# Cost on Machine
m.r = pyo.Param(m.I, initialize={1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 1, 7: 2})
m.d = pyo.Param(m.I, initialize={1: 16, 2: 13, 3: 21, 4: 28, 5: 24, 6: 28, 7: 23})
m.c = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 6, (1, 3): 8,
                                                  (2, 1): 8, (2, 2): 5, (2, 3): 6,
                                                  (3, 1): 12, (3, 2): 7, (3, 3): 10,
                                                  (4, 1): 10, (4, 2): 6, (4, 3): 8,
                                                  (5, 1): 8, (5, 2): 5, (5, 3): 7,
                                                  (6, 1): 12, (6, 2): 7, (6, 3): 10,
                                                  (7, 1): 12, (7, 2): 7, (7, 3): 10})


# Durations p_im for Set 1
# m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 14, (1, 3): 12,
#                                                (2, 1): 6, (2, 2): 8, (2, 3): 7,
#                                                (3, 1): 11, (3, 2): 16, (3, 3): 13,
#                                                (4, 1): 6, (4, 2): 12, (4, 3): 8,
#                                                (5, 1): 10, (5, 2): 16, (5, 3): 12,
#                                                (6, 1): 7, (6, 2): 12, (6, 3): 10,
#                                                (7, 1): 10, (7, 2): 8, (7, 3): 10})

# Uncomment the following and adjust the values for the second set of durations p_im for Set 2

m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,
                                               (2, 1): 3, (2, 2): 4, (2, 3): 3,
                                               (3, 1): 2, (3, 2): 4, (3, 3): 3,
                                               (4, 1): 3, (4, 2): 6, (4, 3): 4,
                                               (5, 1): 2, (5, 2): 4, (5, 3): 3,
                                               (6, 1): 1, (6, 2): 3, (6, 3): 2,
                                               (7, 1): 1, (7, 2): 2, (7, 3): 1})

# Decision Variables
m.y = pyo.Var(m.I, m.J, within=pyo.Binary)  # Job assignment to machine
m.s = pyo.Var(m.I, within=pyo.NonNegativeReals)  # Start time of each job

# Objective Function: Minimize the total cost
def obj_rule(m):
    return sum(m.c[i,j] * m.y[i,j] for i in m.I for j in m.J)

m.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Assignment Constraint: Each job assigned to exactly one machine
def assignment_rule(m, i):
    return sum(m.y[i,j] for j in m.J) == 1

m.assignment_constraint = pyo.Constraint(m.I, rule=assignment_rule)

# Machine Availability and Precedence Constraint: 
# Ensures that for any two jobs i and k, if job i precedes job k, then job k can only start after job i is completed
def precedence_rule(m, i, k, j):
    if i < k:  # Assuming job order determines precedence
        return m.s[i] + sum(m.p[i,jj] * m.y[i,jj] for jj in m.J) <= m.s[k]
    else:
        return pyo.Constraint.Skip

m.precedence_constraint = pyo.Constraint(m.I, m.I, m.J, rule=precedence_rule)

# Release Date Constraint
def release_date_rule(m, i):
    return m.s[i] >= m.r[i]

m.release_date_constraint = pyo.Constraint(m.I, rule=release_date_rule)

# Due Date Constraint
def due_date_rule(m, i):
    return m.s[i] + sum(m.p[i,j] * m.y[i,j] for j in m.J) <= m.d[i]

m.due_date_constraint = pyo.Constraint(m.I, rule=due_date_rule)

solver = pyo.SolverFactory('gurobi')
solver.options['VarBranch'] = 0  # Set to use the most fractional variable branching rule
results = solver.solve(m, tee=True)

# Print the results
print("\nTotal Cost: ", pyo.value(m.objective))
print("\nJob Assignment to Machine:")
for i in m.I:
    for j in m.J:
        if pyo.value(m.y[i,j]) == 1:
            print("Job", i, "assigned to Machine", j, "with start time", pyo.value(m.s[i]))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Read LP format model from file /tmp/tmpv_qpr4u5.pyomo.lp
Reading time = 0.00 seconds
x1: 84 rows, 28 columns, 371 nonzeros
Set parameter VarBranch to value 0
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 84 rows, 28 columns and 371 nonzeros
Model fingerprint: 0x6e7ed9a0
Variable types: 7 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 66.0000000
Presolve removed 77 rows and 15 columns
Presolve time: 0.00s
Presolved: 7 rows, 13 columns, 24 nonzeros
Found heuristic solution: objective 59.0000000
Variable

In [5]:
import pyomo.environ as pyo


m = pyo.ConcreteModel()

# Set declarations (assuming that the 'orders' and 'machines' sets are based on the maximum values from the data)
m.I = pyo.Set(initialize=range(1, 8))  # Set of orders i
m.J = pyo.Set(initialize=range(1, 4))  # Set of machines j

# Parameter declarations
# Cost on Machine
m.r = pyo.Param(m.I, initialize={1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 1, 7: 2})
m.d = pyo.Param(m.I, initialize={1: 16, 2: 13, 3: 21, 4: 28, 5: 24, 6: 28, 7: 23})
m.c = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 6, (1, 3): 8,
                                                  (2, 1): 8, (2, 2): 5, (2, 3): 6,
                                                  (3, 1): 12, (3, 2): 7, (3, 3): 10,
                                                  (4, 1): 10, (4, 2): 6, (4, 3): 8,
                                                  (5, 1): 8, (5, 2): 5, (5, 3): 7,
                                                  (6, 1): 12, (6, 2): 7, (6, 3): 10,
                                                  (7, 1): 12, (7, 2): 7, (7, 3): 10})


# Durations p_im for Set 1
m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 14, (1, 3): 12,
                                               (2, 1): 6, (2, 2): 8, (2, 3): 7,
                                               (3, 1): 11, (3, 2): 16, (3, 3): 13,
                                               (4, 1): 6, (4, 2): 12, (4, 3): 8,
                                               (5, 1): 10, (5, 2): 16, (5, 3): 12,
                                               (6, 1): 7, (6, 2): 12, (6, 3): 10,
                                               (7, 1): 10, (7, 2): 8, (7, 3): 10})


# Decision Variables
m.y = pyo.Var(m.I, m.J, within=pyo.Binary)  # Job assignment to machine
m.s = pyo.Var(m.I, within=pyo.NonNegativeReals)  # Start time of each job

# Objective Function: Minimize the total cost
def obj_rule(m):
    return sum(m.c[i,j] * m.y[i,j] for i in m.I for j in m.J)

m.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Assignment Constraint: Each job assigned to exactly one machine
def assignment_rule(m, i):
    return sum(m.y[i,j] for j in m.J) == 1

m.assignment_constraint = pyo.Constraint(m.I, rule=assignment_rule)

# Machine Availability and Precedence Constraint: 
# Ensures that for any two jobs i and k, if job i precedes job k, then job k can only start after job i is completed
def precedence_rule(m, i, k, j):
    if i < k:  # Assuming job order determines precedence
        return m.s[i] + sum(m.p[i,jj] * m.y[i,jj] for jj in m.J) <= m.s[k]
    else:
        return pyo.Constraint.Skip

m.precedence_constraint = pyo.Constraint(m.I, m.I, m.J, rule=precedence_rule)

# Release Date Constraint
def release_date_rule(m, i):
    return m.s[i] >= m.r[i]

m.release_date_constraint = pyo.Constraint(m.I, rule=release_date_rule)

# Due Date Constraint
def due_date_rule(m, i):
    return m.s[i] + sum(m.p[i,j] * m.y[i,j] for j in m.J) <= m.d[i]

m.due_date_constraint = pyo.Constraint(m.I, rule=due_date_rule)

solver = pyo.SolverFactory('gurobi')
solver.options['BranchDir'] = 0  # Set to enable strong branching
results = solver.solve(m, tee=True)

# # Print the results
# print("\nTotal Cost: ", pyo.value(m.objective))
# print("\nJob Assignment to Machine:")
# for i in m.I:
#     for j in m.J:
#         if pyo.value(m.y[i,j]) == 1:
#             print("Job", i, "assigned to Machine", j, "with start time", pyo.value(m.s[i]))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Read LP format model from file /tmp/tmp9n6c4jie.pyomo.lp
Reading time = 0.00 seconds
x1: 84 rows, 28 columns, 371 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 84 rows, 28 columns and 371 nonzeros
Model fingerprint: 0xbee3aef8
Variable types: 7 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 8 rows and 11 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, b

In [6]:
import pyomo.environ as pyo


m = pyo.ConcreteModel()

# Set declarations (assuming that the 'orders' and 'machines' sets are based on the maximum values from the data)
m.I = pyo.Set(initialize=range(1, 8))  # Set of orders i
m.J = pyo.Set(initialize=range(1, 4))  # Set of machines j

# Parameter declarations
# Cost on Machine
m.r = pyo.Param(m.I, initialize={1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 1, 7: 2})
m.d = pyo.Param(m.I, initialize={1: 16, 2: 13, 3: 21, 4: 28, 5: 24, 6: 28, 7: 23})
m.c = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 6, (1, 3): 8,
                                                  (2, 1): 8, (2, 2): 5, (2, 3): 6,
                                                  (3, 1): 12, (3, 2): 7, (3, 3): 10,
                                                  (4, 1): 10, (4, 2): 6, (4, 3): 8,
                                                  (5, 1): 8, (5, 2): 5, (5, 3): 7,
                                                  (6, 1): 12, (6, 2): 7, (6, 3): 10,
                                                  (7, 1): 12, (7, 2): 7, (7, 3): 10})


# Durations p_im for Set 1
# m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 10, (1, 2): 14, (1, 3): 12,
#                                                (2, 1): 6, (2, 2): 8, (2, 3): 7,
#                                                (3, 1): 11, (3, 2): 16, (3, 3): 13,
#                                                (4, 1): 6, (4, 2): 12, (4, 3): 8,
#                                                (5, 1): 10, (5, 2): 16, (5, 3): 12,
#                                                (6, 1): 7, (6, 2): 12, (6, 3): 10,
#                                                (7, 1): 10, (7, 2): 8, (7, 3): 10})

# Uncomment the following and adjust the values for the second set of durations p_im for Set 2

m.p = pyo.Param(m.I, m.J, initialize={(1, 1): 5, (1, 2): 7, (1, 3): 6,
                                               (2, 1): 3, (2, 2): 4, (2, 3): 3,
                                               (3, 1): 2, (3, 2): 4, (3, 3): 3,
                                               (4, 1): 3, (4, 2): 6, (4, 3): 4,
                                               (5, 1): 2, (5, 2): 4, (5, 3): 3,
                                               (6, 1): 1, (6, 2): 3, (6, 3): 2,
                                               (7, 1): 1, (7, 2): 2, (7, 3): 1})

# Decision Variables
m.y = pyo.Var(m.I, m.J, within=pyo.Binary)  # Job assignment to machine
m.s = pyo.Var(m.I, within=pyo.NonNegativeReals)  # Start time of each job

# Objective Function: Minimize the total cost
def obj_rule(m):
    return sum(m.c[i,j] * m.y[i,j] for i in m.I for j in m.J)

m.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Assignment Constraint: Each job assigned to exactly one machine
def assignment_rule(m, i):
    return sum(m.y[i,j] for j in m.J) == 1

m.assignment_constraint = pyo.Constraint(m.I, rule=assignment_rule)

# Machine Availability and Precedence Constraint: 
# Ensures that for any two jobs i and k, if job i precedes job k, then job k can only start after job i is completed
def precedence_rule(m, i, k, j):
    if i < k:  # Assuming job order determines precedence
        return m.s[i] + sum(m.p[i,jj] * m.y[i,jj] for jj in m.J) <= m.s[k]
    else:
        return pyo.Constraint.Skip

m.precedence_constraint = pyo.Constraint(m.I, m.I, m.J, rule=precedence_rule)

# Release Date Constraint
def release_date_rule(m, i):
    return m.s[i] >= m.r[i]

m.release_date_constraint = pyo.Constraint(m.I, rule=release_date_rule)

# Due Date Constraint
def due_date_rule(m, i):
    return m.s[i] + sum(m.p[i,j] * m.y[i,j] for j in m.J) <= m.d[i]

m.due_date_constraint = pyo.Constraint(m.I, rule=due_date_rule)

solver = pyo.SolverFactory('gurobi')
solver.options['BranchDir'] = 0  # Set to enable strong branching
results = solver.solve(m, tee=True)

# Print the results
print("\nTotal Cost: ", pyo.value(m.objective))
print("\nJob Assignment to Machine:")
for i in m.I:
    for j in m.J:
        if pyo.value(m.y[i,j]) == 1:
            print("Job", i, "assigned to Machine", j, "with start time", pyo.value(m.s[i]))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Read LP format model from file /tmp/tmpuczd0src.pyomo.lp
Reading time = 0.00 seconds
x1: 84 rows, 28 columns, 371 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 84 rows, 28 columns and 371 nonzeros
Model fingerprint: 0x6e7ed9a0
Variable types: 7 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 66.0000000
Presolve removed 77 rows and 15 columns
Presolve time: 0.00s
Presolved: 7 rows, 13 columns, 24 nonzeros
Found heuristic solution: objective 59.0000000
Variable types: 0 continuous, 13 integer (1